In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pyswarm import pso

In [2]:
data = pd.read_csv('training.csv', header=None)
data.head()

,0,1,2,3,4
0,-0.145,-0.145,-0.145,-0.145,-0.145
1,-0.145,-0.145,-0.145,-0.145,-0.145
2,-0.145,-0.145,-0.135,-0.145,-0.120
3,-0.135,-0.145,-0.160,-0.155,-0.150
4,-0.160,-0.155,-0.175,-0.180,-0.160


In [3]:
# Fungsi objektif untuk PSO (menggunakan MSE sebagai objektif)
def objective_function(params):
    # Membangun model jaringan saraf tiruan
    model = keras.Sequential([
        keras.layers.Dense(10, input_dim=data.shape[1] - 1, activation='relu'),
        keras.layers.Dense(1, activation='linear')
    ])
    
    # Set parameter berdasarkan hasil optimasi PSO
    num_input_units = data.shape[1] - 1
    num_hidden_units = 10
    num_output_units = 1
    
    num_weights_layer1 = num_input_units * num_hidden_units
    num_bias_layer1 = num_hidden_units
    num_weights_layer2 = num_hidden_units * num_output_units
    num_bias_layer2 = num_output_units
    
    weights_layer1 = params[:num_weights_layer1].reshape((num_input_units, num_hidden_units))
    bias_layer1 = params[num_weights_layer1:num_weights_layer1 + num_bias_layer1]
    weights_layer2 = params[-num_weights_layer2 - num_bias_layer2:-num_bias_layer2].reshape((num_hidden_units, num_output_units))
    bias_layer2 = params[-num_bias_layer2:]
    
    model.layers[0].set_weights([weights_layer1, bias_layer1])
    model.layers[1].set_weights([weights_layer2, bias_layer2])
    
    # Kompilasi model
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # Menggunakan data yang telah Anda baca dari berkas CSV
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    
    # Latih model
    model.fit(X, y, epochs=100, verbose=0)
    
    # Evaluasi model dan mengembalikan nilai yang ingin dioptimasi
    loss = model.evaluate(X, y, verbose=0)
    return loss

# Batasan parameter untuk PSO
num_input_units = data.shape[1] - 1
num_hidden_units = 10
num_output_units = 1

num_weights_layer1 = num_input_units * num_hidden_units
num_bias_layer1 = num_hidden_units
num_weights_layer2 = num_hidden_units * num_output_units
num_bias_layer2 = num_output_units

num_parameters = num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2
lb = [-1.0] * num_parameters  # Batas bawah parameter
ub = [1.0] * num_parameters   # Batas atas parameter

# Jalankan PSO untuk mengoptimasi parameter jaringan
best_params, _ = pso(objective_function, lb, ub, swarmsize=10, maxiter=50)

# Bangun model jaringan saraf tiruan dengan parameter terbaik
best_model = keras.Sequential([
    keras.layers.Dense(10, input_dim=num_input_units, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])
weights_layer1 = best_params[:num_weights_layer1].reshape((num_input_units, num_hidden_units))
bias_layer1 = best_params[num_weights_layer1:num_weights_layer1 + num_bias_layer1]
weights_layer2 = best_params[-num_weights_layer2 - num_bias_layer2:-num_bias_layer2].reshape((num_hidden_units, num_output_units))
bias_layer2 = best_params[-num_bias_layer2:]
best_model.layers[0].set_weights([weights_layer1, bias_layer1])
best_model.layers[1].set_weights([weights_layer2, bias_layer2])

KeyboardInterrupt: 

In [ ]:
# Simpan model terbaik ke dalam berkas H5
best_model.save("ann_pso.h5")